In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 40.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f298154b4110552ab4434d77da70fa8ff8625d77bd73d882adb5f2b6e77426bf
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [356]:
from pyspark import SparkContext
sc = SparkContext()

In [357]:
movies = sc.textFile('movies.csv')
headerM = movies.first()
movies = movies.filter(lambda x: x != headerM)
movies = movies.map(lambda x: x.split(","))
movies.take(10)

[['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy'],
 ['6', 'Heat (1995)', 'Action|Crime|Thriller'],
 ['7', 'Sabrina (1995)', 'Comedy|Romance'],
 ['8', 'Tom and Huck (1995)', 'Adventure|Children'],
 ['9', 'Sudden Death (1995)', 'Action'],
 ['10', 'GoldenEye (1995)', 'Action|Adventure|Thriller']]

In [358]:
moviesRDD = movies.filter(lambda x: 'Drama' in x[2])
print("No. of Drama movies: %s" % moviesRDD.count())

No. of Drama movies: 3310


In [359]:
ratings = sc.textFile('ratings.csv')
header = ratings.first()
ratings = ratings.filter(lambda x: x != header)
ratings = ratings.map(lambda x: x.split(","))
ratings.take(10)

[['1', '1', '4.0', '964982703'],
 ['1', '3', '4.0', '964981247'],
 ['1', '6', '4.0', '964982224'],
 ['1', '47', '5.0', '964983815'],
 ['1', '50', '5.0', '964982931'],
 ['1', '70', '3.0', '964982400'],
 ['1', '101', '5.0', '964980868'],
 ['1', '110', '4.0', '964982176'],
 ['1', '151', '5.0', '964984041'],
 ['1', '157', '5.0', '964984100']]

In [360]:
ratingsRDD = ratings.map(lambda x: x[1]).distinct()
print("No. of Rated movies:  %s " % ratingsRDD.count())

No. of Rated movies:  9724 


In [361]:
moviesRDD = movies.map(lambda x: x[0])
subtractRDD = moviesRDD.subtract(ratingsRDD)
print("No. of unrated movies %s" % subtractRDD.count())

No. of unrated movies 18


In [362]:
ratingsRDD = ratings.map(lambda x: (x[0], 1))
ratingsRDD = ratingsRDD.reduceByKey(lambda x, y: (x + y))
ratingsRDD = ratingsRDD.sortBy(lambda x: x[1], False)
print("User with most ratings is %s with %s ratings" % (ratingsRDD.first()[0] , ratingsRDD.first()[1]))

User with most ratings is 414 with 2698 ratings


In [363]:
ratingsRDD = ratings.map(lambda x: (x[0], x[2]))

In [366]:
minimum = ratingsRDD.reduceByKey(min)
print("Minimum Ratings per movie: ")
minimum.take(10)

Minimum Ratings per movie: 


[('1', '1.0'),
 ('4', '1.0'),
 ('8', '1.0'),
 ('9', '1.0'),
 ('10', '0.5'),
 ('12', '3.0'),
 ('14', '1.0'),
 ('16', '1.5'),
 ('17', '3.0'),
 ('19', '1.0')]

In [367]:
maximum = ratingsRDD.reduceByKey(max)
print("Maximum Ratings per movie: ")
maximum.take(10)

Maximum Ratings per movie: 


[('1', '5.0'),
 ('4', '5.0'),
 ('8', '5.0'),
 ('9', '5.0'),
 ('10', '5.0'),
 ('12', '5.0'),
 ('14', '5.0'),
 ('16', '5.0'),
 ('17', '5.0'),
 ('19', '5.0')]

In [368]:
average = ratingsRDD.mapValues(lambda v: (v, 1)).reduceByKey(lambda a,b: (float(a[0])+float(b[0]), a[1]+b[1])).mapValues(lambda x : x[0]/x[1])
print("Average Ratings per movie: ")
average.take(10)

Average Ratings per movie: 


[('1', 4.366379310344827),
 ('4', 3.5555555555555554),
 ('8', 3.574468085106383),
 ('9', 3.260869565217391),
 ('10', 3.2785714285714285),
 ('12', 4.390625),
 ('14', 3.3958333333333335),
 ('16', 3.7244897959183674),
 ('17', 4.20952380952381),
 ('19', 2.607396870554765)]

In [369]:
tags = sc.textFile("tags.csv")
headerT = tags.first()
tags = tags.filter(lambda x: x != headerT)
tags = tags.map(lambda x: x.split(","))
tags.take(10)

[['2', '60756', 'funny', '1445714994'],
 ['2', '60756', 'Highly quotable', '1445714996'],
 ['2', '60756', 'will ferrell', '1445714992'],
 ['2', '89774', 'Boxing story', '1445715207'],
 ['2', '89774', 'MMA', '1445715200'],
 ['2', '89774', 'Tom Hardy', '1445715205'],
 ['2', '106782', 'drugs', '1445715054'],
 ['2', '106782', 'Leonardo DiCaprio', '1445715051'],
 ['2', '106782', 'Martin Scorsese', '1445715056'],
 ['7', '48516', 'way too long', '1169687325']]

In [387]:
tagsRDD = tags.map(lambda x: (x[0], x[1])).distinct()
ratingsRDD = ratings.map(lambda x: (x[0], x[1])).distinct()
ratedButNotTagged = ratingsRDD.subtract(tagsRDD)
print("Users with Rated but not Tagged movies: ")
ratedButNotTagged.take(10)

Users with Rated but not Tagged movies: 


[('1', '50'),
 ('1', '70'),
 ('1', '110'),
 ('1', '157'),
 ('1', '216'),
 ('1', '333'),
 ('1', '356'),
 ('1', '441'),
 ('1', '552'),
 ('1', '553')]

In [388]:
print("Users with both Rated and Tagged movies: ")
ratedAndTagged = ratingsRDD.intersection(tagsRDD)
ratedAndTagged.take(10)

Users with both Rated and Tagged movies: 


[('2', '106782'),
 ('18', '44665'),
 ('18', '52604'),
 ('62', '3114'),
 ('62', '6564'),
 ('62', '7153'),
 ('62', '33162'),
 ('62', '38061'),
 ('62', '59501'),
 ('62', '63992')]

In [372]:
moviesRDD = movies.map(lambda x: [x[1][len(x[1]) - 5: len(x[1]) - 1], x[2].split("|")]).filter(lambda x : x[0].isnumeric() and len(x[0]) == 4)
moviesRDD = moviesRDD.flatMapValues(lambda v : v).map(lambda x: (x, 1))
moviesRDD =  moviesRDD.reduceByKey(lambda x, y: (x + y))
print("Number of movies per genre per year: ")
moviesRDD.take(10)

Number of movies per genre per year: 


[(('1995', 'Children'), 20),
 (('1995', 'Fantasy'), 11),
 (('1995', 'Romance'), 38),
 (('1995', 'Drama'), 96),
 (('1995', 'Action'), 39),
 (('1995', 'Thriller'), 34),
 (('1995', 'Horror'), 15),
 (('1995', 'Sci-Fi'), 17),
 (('1995', 'Musical'), 2),
 (('1994', 'Drama'), 87)]

In [355]:
sc.stop()